In [5]:
# import required libraries
import os, sys
import numpy as np, pandas as pd, seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.metrics import r2_score
from scipy.spatial.transform import Rotation

In [12]:
# function to get alignment tensor from RAMAH output files (*.stat)
def get_alignment_tensor(filename):
    fin = open(filename, "r")
    Szz = None
    Sxx = None
    Syy = None
    for line in fin:
        if "Sxx" in line:
            words = line.strip().split('|')
            Sxx = float(words[1].strip())
        elif "Syy" in line:
            words = line.strip().split('|')
            Syy = float(words[1].strip())
        elif "Szz" in line:
            words = line.strip().split('|')
            Szz = float(words[1].strip())
    return (Sxx, Syy, Szz)

In [13]:
# function to get Euler angles from RAMAH output files (*.stat)
def get_euler_angles(filename):
    fin = open(filename, "r")
    alpha = None
    beta = None
    gamma = None
    for line in fin:
        if "Euler Angles" in line:
            line = fin.readline()
            words = line.strip().split()
            alpha = float(words[4])
            beta = float(words[5])
            gamma = float(words[6])
            return (alpha, beta, gamma)

In [15]:
# Read in all relevant alignment tensors and euler angles:
E0_tensor = get_alignment_tensor("data/E0-wt-TAR-H2-Fit.stat")
EI22_tensor = get_alignment_tensor("data/EI22-wt-TAR-H2-Fit.stat")
EII3_tensor = get_alignment_tensor("data/EII3-wt-TAR-All-H2-Fit.stat")
EII13_tensor = get_alignment_tensor("data/EII13-wt-TAR-All-H2-Fit.stat")
E0_abg = get_euler_angles("data/E0-wt-TAR-H2-Fit.stat")
EI22_abg = get_euler_angles("data/EI22-wt-TAR-H2-Fit.stat")
EII3_abg = get_euler_angles("data/EII3-wt-TAR-All-H2-Fit.stat")
EII13_abg = get_euler_angles("data/EII13-wt-TAR-All-H2-Fit.stat")

In [16]:
# function to convert euler angle to SO3 (rotation matrix representation)
def eul2rot(theta) :
    R = np.array([[np.cos(theta[1])*np.cos(theta[2]),       np.sin(theta[0])*np.sin(theta[1])*np.cos(theta[2]) - np.sin(theta[2])*np.cos(theta[0]),      np.sin(theta[1])*np.cos(theta[0])*np.cos(theta[2]) + np.sin(theta[0])*np.sin(theta[2])],
                  [np.sin(theta[2])*np.cos(theta[1]),       np.sin(theta[0])*np.sin(theta[1])*np.sin(theta[2]) + np.cos(theta[0])*np.cos(theta[2]),      np.sin(theta[1])*np.sin(theta[2])*np.cos(theta[0]) - np.sin(theta[0])*np.cos(theta[2])],
                  [-np.sin(theta[1]),                       np.sin(theta[0])*np.cos(theta[1]),                                                           np.cos(theta[0])*np.cos(theta[1])]])

    return R

In [ ]:
###### VERSION 1 ########

In [43]:
# convert alignment tensors from PAS to a shared molecular axis frame:

# E0
R = eul2rot(E0_abg)
Rinv = np.linalg.inv(R)
E0_tensor_pas = np.diag(E0_tensor)
E0_tensor_mol = np.matmul(Rinv, np.matmul(E0_tensor_pas, R))

# EI22
R = eul2rot(EI22_abg)
Rinv = np.linalg.inv(R)
EI22_tensor_pas = np.diag(EI22_tensor)
EI22_tensor_mol = np.matmul(Rinv, np.matmul(EI22_tensor_pas, R))

# EII3
R = eul2rot(EII3_abg)
Rinv = np.linalg.inv(R)
EII3_tensor_pas = np.diag(EII3_tensor)
EII3_tensor_mol = np.matmul(Rinv, np.matmul(EII3_tensor_pas, R))

# EII13
R = eul2rot(EII13_abg)
Rinv = np.linalg.inv(R)
EII13_tensor_pas = np.diag(EII13_tensor)
EII13_tensor_mol = np.matmul(Rinv, np.matmul(EII13_tensor_pas, R))

In [47]:
# function to define normalized scalar product:
def dot(a, b):
    sum = 0
    for i in range(3):
        for j in range(3):
            sum += a[i,j] * b[i,j]
    return sum
def norm_scal_prod(A1, A2):
    num = dot(A1, A2)
    norm1 = np.sqrt(dot(A1, A1)) 
    norm2 = np.sqrt(dot(A2, A2))
    return num/(norm1*norm2)

In [48]:
print("Tensor scalar products VERSION 1:")
print("E0   --- EI22  = %3.2f"%(norm_scal_prod(E0_tensor_mol, EI22_tensor_mol)))
print("     --- EII3  = %3.2f"%(norm_scal_prod(E0_tensor_mol, EII3_tensor_mol)))
print("     --- EII13 = %3.2f"%(norm_scal_prod(E0_tensor_mol, EII13_tensor_mol)))
print("EI22 --- EII3  = %3.2f"%(norm_scal_prod(EI22_tensor_mol, EII3_tensor_mol)))
print("     --- EII13 = %3.2f"%(norm_scal_prod(EI22_tensor_mol, EII13_tensor_mol)))
print("EII3 --- EII13  = %3.2f"%(norm_scal_prod(EII3_tensor_mol, EII13_tensor_mol)))

Tensor scalar products VERSION 1:
E0   --- EI22  = 0.32
     --- EII3  = -0.45
     --- EII13 = -0.42
EI22 --- EII3  = 0.24
     --- EII13 = -0.05
EII3 --- EII13  = -0.00


In [49]:
###### VERSION 2 ########
# convert alignment tensors from PAS to a shared molecular axis frame:

# E0
R = eul2rot(E0_abg)
Rinv = np.linalg.inv(R)
E0_tensor_pas = np.diag(E0_tensor)
E0_tensor_mol = np.matmul(R, np.matmul(E0_tensor_pas, Rinv))

# EI22
R = eul2rot(EI22_abg)
Rinv = np.linalg.inv(R)
EI22_tensor_pas = np.diag(EI22_tensor)
EI22_tensor_mol = np.matmul(R, np.matmul(EI22_tensor_pas, Rinv))

# EII3
R = eul2rot(EII3_abg)
Rinv = np.linalg.inv(R)
EII3_tensor_pas = np.diag(EII3_tensor)
EII3_tensor_mol = np.matmul(R, np.matmul(EII3_tensor_pas, Rinv))

# EII13
R = eul2rot(EII13_abg)
Rinv = np.linalg.inv(R)
EII13_tensor_pas = np.diag(EII13_tensor)
EII13_tensor_mol = np.matmul(R, np.matmul(EII13_tensor_pas, Rinv))

print("Tensor scalar products VERSION 2:")
print("E0   --- EI22  = %3.2f"%(norm_scal_prod(E0_tensor_mol, EI22_tensor_mol)))
print("     --- EII3  = %3.2f"%(norm_scal_prod(E0_tensor_mol, EII3_tensor_mol)))
print("     --- EII13 = %3.2f"%(norm_scal_prod(E0_tensor_mol, EII13_tensor_mol)))
print("EI22 --- EII3  = %3.2f"%(norm_scal_prod(EI22_tensor_mol, EII3_tensor_mol)))
print("     --- EII13 = %3.2f"%(norm_scal_prod(EI22_tensor_mol, EII13_tensor_mol)))
print("EII3 --- EII13  = %3.2f"%(norm_scal_prod(EII3_tensor_mol, EII13_tensor_mol)))

Tensor scalar products VERSION 2:
E0   --- EI22  = 0.32
     --- EII3  = -0.45
     --- EII13 = -0.42
EI22 --- EII3  = 0.24
     --- EII13 = -0.05
EII3 --- EII13  = -0.00
